In [ ]:
%cd ~/Documents/comp3610A3/
import polars as pl
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse._matrix import spmatrix
from amazon.models import *
import polars as pl
import numpy as np
import joblib
from plotly.graph_objs._figure import Figure

In [ ]:
query: pl.Expr = (pl.col("rating") > 3).cast(pl.Int8).alias("rating")

lf: pl.LazyFrame = pl.scan_parquet("data/processed/amazon-2023.parquet")
lf = lf.select(["rating", "text"])
df: pl.DataFrame = lf.with_columns(query).collect()

In [ ]:
X: pl.Series = df["text"]
y: pl.Series = df["rating"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, shuffle=True
)

In [ ]:
vectorizer = TfidfVectorizer(
    lowercase=True,
    token_pattern=r'\b\w+\b', 
    min_df=5,
    max_df=0.8,
    stop_words="english"
)

X_train: spmatrix = vectorizer.fit_transform(X_train)
X_test: spmatrix = vectorizer.transform(X_test)

model = LogisticRegression(max_iter=1_000) # iter added to stop warning
model.fit(X_train, y_train)

In [ ]:
joblib.dump(model, "data/models/logreg_tfidf.pkl")
y_pred: np.ndarray = model.predict(X_test)
metrics: dict = get_metrics(y_test, y_pred, "Logistic Regression")

result = pl.DataFrame([metrics])
result

In [ ]:
fig: Figure = plot_confusion_matrix(result)
fig.show()